<a href="https://colab.research.google.com/github/Changho0514/web1/blob/main/%5BQUS%5D_How_to_use_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sequence를 처리하기 위한 RNN 

1. 주어진 데이터를 RNN에 넣을 수 있는 형태로 만듭니다.
2. 기본적인 RNN 사용법 및 적용법을 익힙니다.
3. LSTM, GRU의 사용법 및 적용법을 익힙니다.

In [2]:
from tqdm import tqdm
from torch import nn
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
import torch

## 데이터 전처리

아래의 sample data를 확인해봅시다.  
전체 단어 수와 pad token의 id도 아래와 같습니다.

In [3]:
vocab_size = 100
pad_id = 0

data = [
  [85,14,80,34,99,20,31,65,53,86,3,58,30,4,11,6,50,71,74,13],
  [62,76,79,66,32],
  [93,77,16,67,46,74,24,70],
  [19,83,88,22,57,40,75,82,4,46],
  [70,28,30,24,76,84,92,76,77,51,7,20,82,94,57],
  [58,13,40,61,88,18,92,89,8,14,61,67,49,59,45,12,47,5],
  [22,5,21,84,39,6,9,84,36,59,32,30,69,70,82,56,1],
  [94,21,79,24,3,86],
  [80,80,33,63,34,63],
  [87,32,79,65,2,96,43,80,85,20,41,52,95,50,35,96,24,80]
]

In [4]:
data[0]

[85, 14, 80, 34, 99, 20, 31, 65, 53, 86, 3, 58, 30, 4, 11, 6, 50, 71, 74, 13]

Padding 처리를 해주면서 padding 전 길이도 저장합니다.

In [5]:
max_len = len(max(data, key=len))
print(f"Maximum sequence length: {max_len}")

valid_lens = []
for i, seq in enumerate(tqdm(data)):
  valid_lens.append(len(seq))
  if len(seq) < max_len:
    data[i] = seq + [pad_id] * (max_len - len(seq))

Maximum sequence length: 20


100%|██████████| 10/10 [00:00<00:00, 42755.39it/s]


In [6]:
print(data)
print(valid_lens)

[[85, 14, 80, 34, 99, 20, 31, 65, 53, 86, 3, 58, 30, 4, 11, 6, 50, 71, 74, 13], [62, 76, 79, 66, 32, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [93, 77, 16, 67, 46, 74, 24, 70, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [19, 83, 88, 22, 57, 40, 75, 82, 4, 46, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [70, 28, 30, 24, 76, 84, 92, 76, 77, 51, 7, 20, 82, 94, 57, 0, 0, 0, 0, 0], [58, 13, 40, 61, 88, 18, 92, 89, 8, 14, 61, 67, 49, 59, 45, 12, 47, 5, 0, 0], [22, 5, 21, 84, 39, 6, 9, 84, 36, 59, 32, 30, 69, 70, 82, 56, 1, 0, 0, 0], [94, 21, 79, 24, 3, 86, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [80, 80, 33, 63, 34, 63, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [87, 32, 79, 65, 2, 96, 43, 80, 85, 20, 41, 52, 95, 50, 35, 96, 24, 80, 0, 0]]
[20, 5, 8, 10, 15, 18, 17, 6, 6, 18]


In [7]:
# B: batch size, L: maximum sequence length
batch = torch.LongTensor(data)  # (B, L)
batch_lens = torch.LongTensor(valid_lens)  # (B)

In [9]:
batch.shape

torch.Size([10, 20])

In [10]:
batch_lens, sorted_idx = batch_lens.sort(descending=True)
batch = batch[sorted_idx]

In [11]:
print(batch)
print(batch_lens)

tensor([[85, 14, 80, 34, 99, 20, 31, 65, 53, 86,  3, 58, 30,  4, 11,  6, 50, 71,
         74, 13],
        [58, 13, 40, 61, 88, 18, 92, 89,  8, 14, 61, 67, 49, 59, 45, 12, 47,  5,
          0,  0],
        [87, 32, 79, 65,  2, 96, 43, 80, 85, 20, 41, 52, 95, 50, 35, 96, 24, 80,
          0,  0],
        [22,  5, 21, 84, 39,  6,  9, 84, 36, 59, 32, 30, 69, 70, 82, 56,  1,  0,
          0,  0],
        [70, 28, 30, 24, 76, 84, 92, 76, 77, 51,  7, 20, 82, 94, 57,  0,  0,  0,
          0,  0],
        [19, 83, 88, 22, 57, 40, 75, 82,  4, 46,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0],
        [93, 77, 16, 67, 46, 74, 24, 70,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0],
        [94, 21, 79, 24,  3, 86,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0],
        [80, 80, 33, 63, 34, 63,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0],
        [62, 76, 79, 66, 32,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0]])
tensor([2

## RNN 사용해보기

RNN에 넣기 전 word embedding을 위한 embedding layer를 만듭니다.

In [15]:
embedding_size = 256

## TODO ##
embedding = nn.Embedding(vocab_size, embedding_size) # (단어들 갯수, word의 vector차원)

# d_w: embedding size
batch_emb = embedding(batch)  # (B, L, d_w)

In [16]:
batch.shape

torch.Size([10, 20])

In [18]:
# 각각의 토큰에 대해서 vector로 만들어 준 것 
batch_emb.shape

torch.Size([10, 20, 256])

RNN 모델 및 초기 hidden state를 정의

- batch_emb 변수를 RNN에 넣을 예정입니다.
- torch 공식 문서를 참조하여, RNN 모델을 정의해보세요. 
- input size는 어떻게 되어야 하나요?

N, L H_in = batch_emb.shape


In [19]:
hidden_size = 512  # RNN의 hidden size
num_layers = 1  # 쌓을 RNN layer의 개수
num_dirs = 1  # 1: 단방향 RNN, 2: 양방향 RNN

rnn = nn.RNN(
    # TODO #
    input_size = embedding_size,
    hidden_size = hidden_size,
    num_layers = num_layers,
    bidirectional = True if num_dirs > 1 else False,
    batch_first = True # batch가 가장 앞으로 나오게 되어있다. 
)

h_0 = torch.zeros((num_layers * num_dirs, batch.shape[0], hidden_size))  # (num_layers * num_dirs, B, d_h)

output, h_n = rnn(batch_emb, h_0)
이렇게 넣으면 안된다.
batch_emb.shape은 batch가 제일 먼저 나온다  
batch_first = False라고 하면 batch_size가 가장 앞에 나와야 함  
따라서 
output, h_n = rnn(batch_emb.reshape(L, N, H_in), h_0) 이런식으로 나와야 사용할 수 있다

### **Vanilla RNN 활용법**

RNN에 batch data를 넣으면 아래와 같이 2가지 output을 얻습니다.


*   `hidden_states`: 각 time step에 해당하는 hidden state들의 묶음.
*   `h_n`: 모든 sequence를 거치고 나온 마지막 hidden state.

torch의 RNN 문서를 참조하여서, ``batch_emb``변수를 rnn에 input으로 넣어보세요.
나온 결과의 shape도 출력해보세요. 

In [21]:
N, L, H_in = batch_emb.shape
hidden_states, h_n = rnn(batch_emb, h_0)

In [31]:
hidden_states, h_n = rnn(batch_emb, h_0)

In [33]:
hidden_states[:,-1,:], h_n.reshape(10, 1, 512)

(tensor([[ 0.4969, -0.4177, -0.4530,  ..., -0.7763,  0.0924,  0.4577],
         [ 0.7106, -0.1318,  0.2029,  ..., -0.0729,  0.6004,  0.5190],
         [ 0.6978, -0.0470,  0.2689,  ..., -0.0362,  0.4559,  0.4272],
         ...,
         [ 0.8446,  0.0419,  0.0241,  ...,  0.1093,  0.4085,  0.5593],
         [ 0.8445,  0.0419,  0.0241,  ...,  0.1093,  0.4085,  0.5593],
         [ 0.8446,  0.0419,  0.0241,  ...,  0.1093,  0.4085,  0.5593]],
        grad_fn=<SliceBackward0>),
 tensor([[[ 0.4969, -0.4177, -0.4530,  ..., -0.7763,  0.0924,  0.4577]],
 
         [[ 0.7106, -0.1318,  0.2029,  ..., -0.0729,  0.6004,  0.5190]],
 
         [[ 0.6978, -0.0470,  0.2689,  ..., -0.0362,  0.4559,  0.4272]],
 
         ...,
 
         [[ 0.8446,  0.0419,  0.0241,  ...,  0.1093,  0.4085,  0.5593]],
 
         [[ 0.8445,  0.0419,  0.0241,  ...,  0.1093,  0.4085,  0.5593]],
 
         [[ 0.8446,  0.0419,  0.0241,  ...,  0.1093,  0.4085,  0.5593]]],
        grad_fn=<ReshapeAliasBackward0>))

마지막 hidden state를 이용하여 text classification task에 적용할 수 있습니다.

In [34]:
num_classes = 2
classification_layer = nn.Linear(hidden_size, num_classes)

# C: number of classes
output = classification_layer(h_n.squeeze(0))  # (1, B, d_h) => (B, C)
print(output.shape)

torch.Size([10, 2])


각 time step에 대한 hidden state를 이용하여 token-level의 task를 수행할 수도 있습니다.

In [36]:
num_classes = 5
entity_layer = nn.Linear(hidden_size, num_classes)

# C: number of classes
output = entity_layer(hidden_states)  # (L, B, d_h) => (L, B, C)
print(output.shape)

torch.Size([10, 20, 5])


### **LSTM 활용법**

LSTM에선 cell state가 추가됩니다.  
Cell state의 shape는 hidden state의 그것과 동일합니다.

- batch_emb 변수를 LSTM에 넣을 예정입니다.
- torch 공식 문서를 참조하여, LSTM 모델을 정의해보세요. 
- input size는 어떻게 되어야 하나요?

In [37]:
hidden_size = 512
num_layers = 1
num_dirs = 1

## TODO ##
lstm = nn.LSTM(
    input_size = embedding_size,
    hidden_size = hidden_size,
    num_layers = num_layers,
    bidirectional = True if num_dirs > 1 else False,
    batch_first = True
)

h_0 = torch.zeros((num_layers * num_dirs, batch.shape[0], hidden_size))  # (num_layers * num_dirs, B, d_h)
c_0 = torch.zeros((num_layers * num_dirs, batch.shape[0], hidden_size))  # (num_layers * num_dirs, B, d_h)

torch의 LSTM 문서를 참조하여서, ``batch_emb``변수를 rnn에 input으로 넣어보세요.
나온 결과의 shape도 출력해보세요. 

In [38]:
## TODO ##
output, (h_n, c_n) =lstm(batch_emb)

In [39]:
output.shape, h_n.shape, c_n.shape


output[:, -1,:] #output의 제일 뒷부분이 필요할 경우

tensor([[ 0.0322,  0.1457, -0.0418,  ..., -0.0330,  0.0359, -0.0761],
        [ 0.1041, -0.0197, -0.1661,  ...,  0.2366,  0.0287, -0.1752],
        [ 0.1793, -0.0448, -0.1474,  ...,  0.2302, -0.1761, -0.1182],
        ...,
        [ 0.1961, -0.1456, -0.1974,  ...,  0.2676, -0.1352, -0.2442],
        [ 0.1963, -0.1457, -0.1973,  ...,  0.2675, -0.1362, -0.2441],
        [ 0.1959, -0.1450, -0.1975,  ...,  0.2674, -0.1352, -0.2447]],
       grad_fn=<SliceBackward0>)

### **GRU 사용**

GRU는 cell state가 없어 RNN과 동일하게 사용 가능합니다.   
GRU를 이용하여 LM task를 수행해봅시다.

In [40]:
gru = nn.GRU(
    input_size=embedding_size,
    hidden_size=hidden_size,
    num_layers=num_layers,
    bidirectional=True if num_dirs > 1 else False
)

In [41]:
output_layer = nn.Linear(hidden_size, vocab_size)

In [42]:
input_id = batch.transpose(0, 1)[0, :]  # (B)
hidden = torch.zeros((num_layers * num_dirs, batch.shape[0], hidden_size))  # (1, B, d_h)

In [43]:
for t in range(max_len):
  input_emb = embedding(input_id).unsqueeze(0)  # (1, B, d_w)
  output, hidden = gru(input_emb, hidden)  # output: (1, B, d_h), hidden: (1, B, d_h)

  # V: vocab size
  output = output_layer(output)  # (1, B, V)
  probs, top_id = torch.max(output, dim=-1)  # probs: (1, B), top_id: (1, B)

  print("*" * 50)
  print(f"Time step: {t}")
  print(output.shape)
  print(probs.shape)
  print(top_id.shape)

  input_id = top_id.squeeze(0)  # (B)

**************************************************
Time step: 0
torch.Size([1, 10, 100])
torch.Size([1, 10])
torch.Size([1, 10])
**************************************************
Time step: 1
torch.Size([1, 10, 100])
torch.Size([1, 10])
torch.Size([1, 10])
**************************************************
Time step: 2
torch.Size([1, 10, 100])
torch.Size([1, 10])
torch.Size([1, 10])
**************************************************
Time step: 3
torch.Size([1, 10, 100])
torch.Size([1, 10])
torch.Size([1, 10])
**************************************************
Time step: 4
torch.Size([1, 10, 100])
torch.Size([1, 10])
torch.Size([1, 10])
**************************************************
Time step: 5
torch.Size([1, 10, 100])
torch.Size([1, 10])
torch.Size([1, 10])
**************************************************
Time step: 6
torch.Size([1, 10, 100])
torch.Size([1, 10])
torch.Size([1, 10])
**************************************************
Time step: 7
torch.Size([1, 10, 100])
torch.Si

### **양방향 및 여러 layer 사용**

이번엔 양방향 + 2개 이상의 layer를 쓸 때 얻을 수 있는 결과에 대해 알아봅니다.


In [44]:
num_layers = 2
num_dirs = 2
dropout=0.1

gru = nn.GRU(
    input_size=embedding_size,
    hidden_size=hidden_size,
    num_layers=num_layers,
    dropout=dropout,
    bidirectional=True if num_dirs > 1 else False
)

Bidirectional이 되었고 layer의 개수가 $2$로 늘었기 때문에 hidden state의 shape도 `(4, B, d_h)`가 됩니다.

In [45]:
# d_w: word embedding size, num_layers: layer의 개수, num_dirs: 방향의 개수
batch_emb = embedding(batch)  # (B, L, d_w)
h_0 = torch.zeros((num_layers * num_dirs, batch.shape[0], hidden_size))  # (num_layers * num_dirs, B, d_h) = (4, B, d_h)

packed_batch = pack_padded_sequence(batch_emb.transpose(0, 1), batch_lens)

packed_outputs, h_n = gru(packed_batch, h_0)
print(packed_outputs)
print(packed_outputs[0].shape)
print(h_n.shape)

PackedSequence(data=tensor([[-0.0470, -0.1891,  0.1556,  ..., -0.0798,  0.1125, -0.2541],
        [ 0.0282, -0.0565,  0.0718,  ..., -0.1477,  0.1251, -0.0179],
        [-0.0677,  0.1353, -0.0156,  ...,  0.0016,  0.0121,  0.0860],
        ...,
        [-0.2320, -0.1410,  0.0334,  ..., -0.1367,  0.0985, -0.0995],
        [-0.0760, -0.0868,  0.0881,  ..., -0.1147, -0.1219, -0.0702],
        [-0.0783, -0.2514,  0.0836,  ..., -0.1202, -0.0884, -0.0358]],
       grad_fn=<CatBackward0>), batch_sizes=tensor([10, 10, 10, 10, 10,  9,  7,  7,  6,  6,  5,  5,  5,  5,  5,  4,  4,  3,
         1,  1]), sorted_indices=None, unsorted_indices=None)
torch.Size([123, 1024])
torch.Size([4, 10, 512])


In [46]:
outputs, output_lens = pad_packed_sequence(packed_outputs)

print(outputs.shape)  # (L, B, num_dirs*d_h)
print(output_lens)

torch.Size([20, 10, 1024])
tensor([20, 18, 18, 17, 15, 10,  8,  6,  6,  5])


각각의 결과물의 shape는 다음과 같습니다.

`outputs`: `(max_len, batch_size, num_dir * hidden_size)`  
`h_n`: `(num_layers*num_dirs, batch_size, hidden_size)`

In [47]:
batch_size = h_n.shape[1]
print(h_n.view(num_layers, num_dirs, batch_size, hidden_size))
print(h_n.view(num_layers, num_dirs, batch_size, hidden_size).shape)

tensor([[[[-0.2724, -0.2822, -0.1090,  ...,  0.2506, -0.3323,  0.0265],
          [-0.2999,  0.0186,  0.3451,  ..., -0.0824, -0.2832, -0.4517],
          [ 0.0200, -0.2409, -0.2098,  ...,  0.2035, -0.0612, -0.2836],
          ...,
          [-0.0880, -0.3622, -0.1911,  ..., -0.0555,  0.1137, -0.2440],
          [ 0.0398, -0.1484,  0.1893,  ..., -0.4380, -0.0956, -0.4010],
          [-0.1810, -0.2444,  0.0497,  ...,  0.0760,  0.0066,  0.0766]],

         [[-0.2194,  0.0906,  0.1520,  ...,  0.2620, -0.1005, -0.1851],
          [ 0.3343, -0.1336, -0.1695,  ..., -0.3477,  0.0234,  0.1033],
          [ 0.0155, -0.2534,  0.1353,  ...,  0.2593, -0.0909, -0.0084],
          ...,
          [-0.3252,  0.3058, -0.2947,  ...,  0.4062,  0.2628,  0.0749],
          [ 0.4008,  0.2373,  0.5453,  ..., -0.2083, -0.1086, -0.3592],
          [-0.1681,  0.5277, -0.1958,  ..., -0.1763, -0.4486,  0.2234]]],


        [[[-0.0783, -0.2514,  0.0836,  ...,  0.1483, -0.0920,  0.2808],
          [-0.0704, -0.1932,